In [3]:
import pandas as pd
import re

In [7]:
from transliterate import translit

In [13]:
translit('Евгения', "ru", reversed=True)

'Evgenija'

In [11]:
translit('Yevgeniya', "ru")

'Ыевгениыа'

In [71]:
data_init = pd.read_csv('name_gender.csv', names=['surname', 'name', 'thirdname', 'gender'], header=None)

In [96]:
data = data_init

In [97]:
data = data.drop_duplicates()

In [100]:
data = data.dropna()
data = data.loc[data['gender'].isin(('М', 'Ж'))]

In [101]:
data.gender = data.gender.map({'М': 1, 'Ж': 0})


In [124]:
data = data[['surname', 'name', 'gender']]

In [125]:
data.surname = data.surname.apply(lambda x: x.capitalize())
data.name = data.name.apply(lambda x: x.capitalize())

In [111]:
preprocess_str = lambda x: re.sub(r'\s+ ', '', str(x).upper()).replace('Ё', 'Е')
data['name'] = data['name'].apply(preprocess_str)
data['surname'] = data['surname'].apply(preprocess_str)

In [112]:
def is_cirillic(string):
    return bool(re.search('^([А-Я]+\-?[А-Я]+)?$', str(string)))

data = data[data['name'].apply(is_cirillic)]
data = data[data['surname'].apply(is_cirillic)]

In [129]:
def preprocess(surname, name):
    surname_n = surname[-2:]
    name_n = name[-2:]
    
    num_let_sur = len(surname)
    num_let_nam = len(name)
    
    has_m_surend = int(surname[-2:].upper() in set(['OV', 'IN', 'AN']))
    last_let_name = int(name[-1].upper() == 'A')

    return pd.Series([
        surname,
        name,
        surname_n,
        name_n, 
        has_m_surend,
        num_let_sur,
        num_let_nam,
        last_let_name
    ])

In [121]:
data.surname = data.surname.apply(lambda x: translit(x, "ru", reversed=True))
data.name = data.name.apply(lambda x: translit(x, "ru", reversed=True))

C:\Users\Kirill\AppData\Local\Temp\ipykernel_21984\490937923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.surname = data.surname.apply(lambda x: translit(x, "ru", reversed=True))
C:\Users\Kirill\AppData\Local\Temp\ipykernel_21984\490937923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.name = data.name.apply(lambda x: translit(x, "ru", reversed=True))


In [126]:
data

,surname,name,gender
0,Ababij,Andrej,1
1,Ababilov,Dmitrij,1
2,Ababilov,Vladimir,1
3,Ababilov,Sergej,1
4,Ababilov,Jurij,1
...,...,...,...
6228558,Ksenda,Grigorij,1
6228571,Mescherjakova,Alena,0
6228584,Shestoperov,Ivan,1
6228601,Semenova,Efrosin'ja,0


In [128]:
data_prep = data.copy()

In [132]:
data_prep = data_prep[['surname', 'name']].apply(lambda x: preprocess(x['surname'], x['name']), axis=1)

In [136]:
data_prep['target'] = data['gender']

In [138]:
data_prep.to_csv('name_gender_features.csv', index=False)

In [137]:
data_prep

,0,1,2,3,4,5,6,7,target
0,Ababij,Andrej,ij,ej,0,6,6,0,1
1,Ababilov,Dmitrij,ov,ij,1,8,7,0,1
2,Ababilov,Vladimir,ov,ir,1,8,8,0,1
3,Ababilov,Sergej,ov,ej,1,8,6,0,1
4,Ababilov,Jurij,ov,ij,1,8,5,0,1
...,...,...,...,...,...,...,...,...,...
6228558,Ksenda,Grigorij,da,ij,0,6,8,0,1
6228571,Mescherjakova,Alena,va,na,0,13,5,1,0
6228584,Shestoperov,Ivan,ov,an,1,11,4,0,1
6228601,Semenova,Efrosin'ja,va,ja,0,8,10,1,0


In [68]:
data.to_csv('names_by_gender.csv', index=False)

In [4]:
data = pd.read_csv('names_by_gender.csv')

In [5]:
data

,surname,name,gender
0,Ababij,Andrej,1.0
1,Ababilov,Dmitrij,1.0
2,Ababilov,Vladimir,1.0
3,Ababilov,Sergej,1.0
4,Ababilov,Jurij,1.0
...,...,...,...
2103290,Muhotoryh,Aleksandr,1.0
2103291,Tihorenko,Aleksandr,1.0
2103292,Gonsales,Echavarrija,1.0
2103293,Vontobel'-,Kulikova,0.0


In [9]:
data[data.name == 'Evgenija']

,surname,name,gender
57,Abaeva,Evgenija,0.0
173,Abakumova,Evgenija,0.0
235,Abalihina,Evgenija,0.0
245,Abalkina,Evgenija,0.0
296,Abanina,Evgenija,0.0
...,...,...,...
2102989,Garmanova,Evgenija,0.0
2102994,Chebakova,Evgenija,0.0
2103055,Nagaljuk,Evgenija,0.0
2103076,Nichiporchuk,Evgenija,0.0
